In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.getcwd()) 

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test_2.csv')
sample = pd.read_csv('sample_submission_2.csv')

C:\Users\matze\Documents\Winton


In [2]:
#print("Column Names:::", df_train.columns.values.tolist())
print("Columns with missing Values::", df_train.columns[df_train.isnull().any()].tolist())
### columns without missing values
print("Short:")
print(df_train.info())

Columns with missing Values:: ['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_6', 'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13', 'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17', 'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23', 'Feature_24', 'Feature_25', 'Ret_2', 'Ret_3', 'Ret_4', 'Ret_5', 'Ret_6', 'Ret_7', 'Ret_8', 'Ret_9', 'Ret_10', 'Ret_11', 'Ret_12', 'Ret_13', 'Ret_14', 'Ret_15', 'Ret_16', 'Ret_17', 'Ret_18', 'Ret_19', 'Ret_20', 'Ret_21', 'Ret_22', 'Ret_23', 'Ret_24', 'Ret_25', 'Ret_26', 'Ret_27', 'Ret_28', 'Ret_29', 'Ret_30', 'Ret_31', 'Ret_32', 'Ret_33', 'Ret_34', 'Ret_35', 'Ret_36', 'Ret_37', 'Ret_38', 'Ret_39', 'Ret_40', 'Ret_41', 'Ret_42', 'Ret_43', 'Ret_44', 'Ret_45', 'Ret_46', 'Ret_47', 'Ret_48', 'Ret_49', 'Ret_50', 'Ret_51', 'Ret_52', 'Ret_53', 'Ret_54', 'Ret_55', 'Ret_56', 'Ret_57', 'Ret_58', 'Ret_59', 'Ret_60', 'Ret_61', 'Ret_62', 'Ret_63', 'Ret_64', 'Ret_65', 'Ret_66', 'Ret_67', 'R

We've got an overall Trainning DataSet with 40,000 entries and 25 static features, 2 daily and 119 minute returnrates from the past and 2 Daily return rates and 60 minute rates from the future

In the dataset is an ID columns which is unnecessary and can be droped and two Features (5 and 7) with no missing values on which we'll have a special focus
Feature 5 might be categorial so we want to hotencode it




In [3]:
df_train = df_train.drop('Id', axis = 'columns')

In [4]:
df_train.Feature_5 = df_train.Feature_5.astype(int)


In [5]:
Feature_5_dummies = pd.get_dummies(df_train.Feature_5, prefix  = "Feature_5")
df_train = pd.concat([df_train, Feature_5_dummies], axis = 1, verify_integrity = True)

In [6]:
df_train.loc[:, 'Feature_1':'Ret_120'].columns.append(Feature_5_dummies.columns)

Index(['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5',
       'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9', 'Feature_10',
       ...
       'Feature_5_1', 'Feature_5_2', 'Feature_5_3', 'Feature_5_4',
       'Feature_5_5', 'Feature_5_6', 'Feature_5_7', 'Feature_5_8',
       'Feature_5_9', 'Feature_5_10'],
      dtype='object', length=156)

In [7]:
X = df_train.loc[:, (df_train.loc[:, 'Feature_1':'Ret_120'].columns.append(Feature_5_dummies.columns))]
y = df_train.loc[:, 'Ret_121':'Ret_PlusTwo']
weight_daily = df_train.Weight_Daily
weight_intraday = df_train.Weight_Intraday
featureset = df_train.loc[:, 'Feature_1':'Feature_25']
featureset_encoded  = df_train.loc[:, (featureset.columns.append(Feature_5_dummies.columns))]
daily_ret_past = df_train.loc[:, 'Ret_MinusTwo':'Ret_MinusOne']
daily_ret_fut = df_train.loc[:, 'Ret_PlusOne':'Ret_PlusOne']
minute_ret_past = df_train.loc[:, 'Ret_2':'Ret_120']
minute_ret_fut = df_train.loc[:, 'Ret_121':'Ret_180']


In [20]:
weight_daily.shape

(40000,)

## Test Train Split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle = False, random_state = 0)

## Preprocessing

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer

### creating the pipelines

In [10]:
minute_pipeline = Pipeline([
 ('imputer', Imputer(axis=1))
])
feature_pipeline = Pipeline([
 ('imputer', Imputer())
 ])
daily_pipeline = Pipeline([
 ('imputer', Imputer())
 ])

### fit and transform on train

In [11]:
X_train_minute_prep = pd.DataFrame(minute_pipeline.fit_transform(X_train.loc[:, minute_ret_past.columns]), columns= minute_ret_past.columns, index = X_train.index)
X_train_daily_prep = pd.DataFrame(daily_pipeline.fit_transform(X_train.loc[:, daily_ret_past.columns]), columns= daily_ret_past.columns, index = X_train.index)
X_train_feature_prep = pd.DataFrame(feature_pipeline.fit_transform(X_train.loc[:, featureset_encoded.columns]), columns= featureset_encoded.columns, index = X_train.index)

X_train_prep = pd.concat([X_train_feature_prep, X_train_daily_prep, X_train_minute_prep] ,axis='columns', verify_integrity = True)


### transform on test

In [12]:
X_test_minute_prep =  pd.DataFrame(minute_pipeline.transform(X_test.loc[:, minute_ret_past.columns]), columns= minute_ret_past.columns, index = X_test.index)
X_test_daily_prep =  pd.DataFrame(daily_pipeline.transform(X_test.loc[:, daily_ret_past.columns]), columns= daily_ret_past.columns, index = X_test.index)
X_test_feature_prep =  pd.DataFrame(feature_pipeline.transform(X_test.loc[:, featureset_encoded.columns]), columns= featureset_encoded.columns, index = X_test.index)


X_test_prep = pd.concat([X_test_feature_prep, X_test_daily_prep, X_test_minute_prep] ,axis='columns', verify_integrity = True)


## Export cleaned Datasets



In [15]:
#pd.DataFrame.to_csv¶("Clean Data/")
X_train_prep.to_csv("Clean Data/X_train_prep.csv")
X_test_prep.to_csv("Clean Data/X_test_prep.csv")


In [16]:
y_train.to_csv("Clean Data/y_train_prep.csv")
y_test.to_csv("Clean Data/y_test_prep.csv")

In [18]:
weight_daily.to_csv("Clean Data/weight_daily.csv")
weight_intraday.to_csv("Clean Data/weight_intraday.csv")

In [21]:
weight_daily.shape

(40000,)